In [1]:
import flowcode
import processing
import res_flow_vis as visual

import torch
import numpy as np

In [2]:
#Filename associated with this specific run
filename = "testrun2"

In [3]:
#Initiate a processor to handle data
mpc = processing.Processor_cond(N_min=500, percentile2=95)

In [4]:
#Load raw data
Data, N_stars, M_stars, M_dm = mpc.get_data("all_sims")

In [5]:
#Clean data
Data_const, N_stars_const, M_stars_const, M_dm_const = mpc.constraindata(Data, M_dm)

Cut out 0 of 95 galaxies, 4417968 of 34878379 stars (~13%).


In [6]:
#Choose device
device = "cuda"

In [7]:
#Hyperparameters of the flow
LAYER_TYPE = flowcode.NSF_CL2
N_LAYERS = 8
COND_INDS = np.array([10])
DIM_COND = COND_INDS.shape[0]
DIM_NOTCOND = Data_const[0].shape[1] - DIM_COND
SPLIT = 0.5
K = 8
B = 3
BASE_NETWORK = flowcode.MLP
BASE_NETWORK_N_LAYERS = 4
BASE_NETWORK_N_HIDDEN = 16
BASE_NETWORK_LEAKY_RELU_SLOPE = 0.2

SPLIT = {"split":SPLIT} if LAYER_TYPE == flowcode.NSF_CL else {}

In [8]:
#Instantiate the model
model = flowcode.NSFlow(N_LAYERS, DIM_NOTCOND, DIM_COND, LAYER_TYPE, **SPLIT, K=K, B=B, network=BASE_NETWORK, network_args=(BASE_NETWORK_N_HIDDEN,BASE_NETWORK_N_LAYERS,BASE_NETWORK_LEAKY_RELU_SLOPE))
model = model.to(device)

In [9]:
#Training hyperparameters
N_EPOCHS = 3
INIT_LR = 0.0002
BATCH_SIZE = 1024
LOG_LEARN = np.array([10])

In [10]:
#Prepare data for flow
Data_flow = mpc.Data_to_flow(mpc.diststack(Data_const), LOG_LEARN)

In [ ]:
#Save relevant data to the drive for external python file (device needs to be GPU) to do the training in background...
torch.save(Data_flow, "data_cond_trainer.pth")
torch.save(model, "model_cond_trainer.pth")
np.save("params_cond_trainer.npy", np.append(COND_INDS,np.array([N_EPOCHS,INIT_LR,BATCH_SIZE])))
np.save("filename_cond_trainer.npy", filename)

In [11]:
#...OR train here
import time
train_loss_saver = []
start = time.perf_counter()
flowcode.train_flow(model, Data_flow, COND_INDS, N_EPOCHS, lr=INIT_LR, batch_size=BATCH_SIZE, loss_saver=train_loss_saver)
end = time.perf_counter()
torch.save(model.state_dict(), f"{filename}.pth")
np.save(f"loss_{filename}.npy",np.array(train_loss_saver+[end-start]))

Step 0 of 89240, Loss:15.25014877319336, lr=0.0002
Step 100 of 89240, Loss:14.914491596221923, lr=0.00019800897604194966
Step 200 of 89240, Loss:14.58655683517456, lr=0.00019603777296590692
Step 300 of 89240, Loss:14.163960170745849, lr=0.00019408619345261712
Step 400 of 89240, Loss:13.887592811584472, lr=0.00019215404214716232
Step 500 of 89240, Loss:13.570443820953368, lr=0.00019024112563940622
Step 600 of 89240, Loss:13.150003070831298, lr=0.0001883472524446336
Step 700 of 89240, Loss:12.606537399291993, lr=0.00018647223298438246
Step 800 of 89240, Loss:11.904420547485351, lr=0.00018461587956746718
Step 900 of 89240, Loss:11.08151174545288, lr=0.00018277800637119036
Step 1000 of 89240, Loss:10.494283924102783, lr=0.00018095842942274178
Step 1100 of 89240, Loss:9.952074069976806, lr=0.00017915696658078257
Step 1200 of 89240, Loss:9.697899208068847, lr=0.0001773734375172127
Step 1300 of 89240, Loss:9.435991744995118, lr=0.00017560766369912012
Step 1400 of 89240, Loss:9.246514091491699

ValueError: Expected value argument (Tensor of shape (1024, 10)) to be within the support (Real()) of the distribution Normal(loc: torch.Size([10]), scale: torch.Size([10])), but found invalid values:
tensor([[nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan],
        ...,
        [nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan]], device='cuda:0',
       grad_fn=<CatBackward0>)

In [ ]:
#Load in training results:
model.load_state_dict(torch.load(f"{filename}.pth"))
loss_results = np.load(f"loss_{filename}.npy")
loss_results, tot_time = loss_results[:-1], loss_results[-1]

In [ ]:
#Get a sample from the flow

#Set a condition for the sample
condition = mpc.diststack(Data_const)[:,COND_INDS]

#Get sample
flow_sample = mpc.galaxysplit(mpc.sample_to_Data(mpc.sample_Conditional(model, COND_INDS, condition)), N_stars_const)

In [1]:
### Visualize data

In [ ]:
#Get multiple galaxy plot (data)
vs_data = visual.plot_conditional(Data_const, M_dm_const+M_stars_const, type="N", label=filename+"_data")

In [ ]:
#Get multiple galaxy plot (sample)
visual.plot_conditional(Data_const, M_dm_const+M_stars_const, type="N", label=filename, v_pre=vs_data)

In [ ]:
#Get comparison plot of single galaxy

visual.get_result_plots(Data_const[50], flow_sample[50], label=filename)

In [ ]:
visual.loss_plot(loss_results, tot_time, savefig=filename)